In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

import random as rd
import matplotlib.pyplot as plt


In [18]:
"""SVM for MNIST Dataset"""
from joblib import load
mnist = load("C:/Users/MaxB2/Documents/Machine_Is_Learning/mnist_dataset_784_v1.joblib")

X,y = mnist["data"],mnist["target"]
y = y.astype(np.uint8)
X_train, X_test, y_train, y_test = X[:60000],X[60000:],y[:60000],y[60000:] 

from sklearn.model_selection import StratifiedShuffleSplit
SS_split = StratifiedShuffleSplit(n_splits=10, test_size=(1/12), random_state=42)
for train_index, test_index in SS_split.split(X_train,y_train):
    X_train_50k = X_train.loc[train_index]
    y_train_50k = y_train.loc[train_index]
    X_train_10k = X_train.loc[test_index]
    y_train_10k = y_train.loc[test_index]
#   X_train_10k , y_train_10k

In [19]:
X_train_10k = X_train_10k.reset_index(drop=True)
y_train_10k = y_train_10k.reset_index(drop=True)

In [20]:
import pandas as pd

# The proportions of the income_cat groups are defined as the following fractions
total_digit_counts = mnist["target"].value_counts() / len(mnist["target"])
strat_test_value_counts = y_train_10k.value_counts()/ len(X_train_10k)
# The absolute value of their respective errors are described as 

# Convert the Series object to a DataFrame with appropriate column names
df_comparison = pd.DataFrame({
    "Normal Proportion": total_digit_counts.values,
    "Strat 10k Test Proportion": strat_test_value_counts.values,
})
df_comparison

,Normal Proportion,Strat 10k Test Proportion
0,0.112529,0.1124
1,0.104186,0.1044
2,0.102014,0.1022
3,0.099857,0.0992
4,0.099400,0.0992
5,0.098614,0.0988
6,0.098229,0.0986
7,0.097500,0.0974
8,0.097486,0.0974
9,0.090186,0.0904


In [21]:
from sklearn.multiclass import OneVsRestClassifier
#Best Paramaters:  {'svc__estimator__C': 0.07, 'svc__estimator__gamma': 0.1, 'svc__estimator__kernel': 'poly'}
#from sklearn.svm import SVC

svc_ovr = Pipeline([
    ("scaler",StandardScaler()),
    ("svc",OneVsRestClassifier(SVC()))
])

svc_ovr.fit(X_train_10k,y_train_10k)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svc', OneVsRestClassifier(estimator=SVC()))])

In [26]:
# from sklearn.model_selection import GridSearchCV as GS
param_grid = [
    {"svc__estimator__kernel":['poly'],
     "svc__estimator__gamma":[0.1,0.3,0.5,0.6],
     "svc__estimator__C":[0.07,0.1,0.13]},
]
grid_search = GS(svc_ovr,
                 param_grid,
                 cv=3,
                 scoring="accuracy",
                 return_train_score=True
)

grid_search.fit(X_train_10k,y_train_10k) # Takes a while

print("Best Paramaters: ", grid_search.best_params_)
print("")
cvres = grid_search.cv_results_
for mean_score, params in zip (cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_score),params)

Best Paramaters:  {'svc__estimator__C': 0.07, 'svc__estimator__gamma': 0.1, 'svc__estimator__kernel': 'poly'}

nan {'svc__estimator__C': 0.07, 'svc__estimator__gamma': 0.1, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.07, 'svc__estimator__gamma': 0.3, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.07, 'svc__estimator__gamma': 0.5, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.07, 'svc__estimator__gamma': 0.6, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.1, 'svc__estimator__gamma': 0.1, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.1, 'svc__estimator__gamma': 0.3, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.1, 'svc__estimator__gamma': 0.5, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.1, 'svc__estimator__gamma': 0.6, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.13, 'svc__estimator__gamma': 0.1, 'svc__estimator__kernel': 'poly'}
nan {'svc__estimator__C': 0.1

C:\Users\MaxB2\AppData\Local\Temp\ipykernel_10088\3361974589.py:20: RuntimeWarning: invalid value encountered in sqrt
  print(np.sqrt(-mean_score),params)


In [29]:
grid_search.best_estimator_.score(X_train,y_train)

0.9535

In [30]:
grid_search.best_estimator_.score(X_test,y_test)

0.9519